In [ ]:
import cv2
import os
import blur_detector
from matplotlib import pyplot as plt
# Open a connection to the default camera (usually the webcam)
cap = cv2.VideoCapture(0)
blurs = []

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open video stream or file")
    exit()
    
def composite(im1,im2):
    # ! im2 is the depthmap
    background,foreground = im1,im2
    alpha_background = background[:,:,3] / 255.0
    alpha_foreground = foreground[:,:,3] / 255.0

    for color in range(0, 3):
        background[:,:,color] = alpha_foreground * foreground[:,:,color] + \
            alpha_background * background[:,:,color] * (1 - alpha_foreground)

    background[:,:,3] = (1 - (1 - alpha_foreground) * (1 - alpha_background)) * 255

    cv2.imshow("Composited image", background)
    cv2.waitKey(0)

# Read and display frames in a loop
cnt = 0
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If frame reading was not successful, break the loop
    if not ret:
        print("Error: Failed to capture image")
        break

    # Display the resulting frame
    flipped = cv2.flip(frame, 0)
    flipped = cv2.flip(flipped, 1)
    cv2.imshow('Camera Output', flipped)

    # Press 'q' to exit the video display
    
    if cv2.waitKey(1) & 0xFF == ord('c'):
        cv2.imwrite(os.path.join(r"REDACTED",'image{}.jpg'.format(cnt)), flipped)
        cnt += 1
        # break
        
    if cv2.waitKey(1) & 0xFF == ord('b'):
        print(flipped.shape)
        gray = cv2.cvtColor(flipped, cv2.COLOR_BGR2GRAY)
        blur_map = blur_detector.detectBlur(gray, downsampling_factor=4, num_scales=4, scale_start=2, num_iterations_RF_filter=3, show_progress=True)
        blurs.append(blur_map)
        # image = cv2.cvtColor(flipped, cv2.COLOR_BGR2RGB)
        image = flipped
        fig, ax = plt.subplots()
        ax.imshow(image)
        ax.imshow(blur_map, alpha=0.5)
        ax.axis('off')

        # output_path = os.path.join(r"C:\Users\victo\Desktop\Files\Tech\Code\Python\neurips\captures",'image{}.jpg'.format(cnt))
        cv2.imwrite(os.path.join(r"REDACTED",'image{}.jpg'.format(cnt)), flipped)
        plt.show()
        cnt += 1
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import os
import blur_detector
from matplotlib import pyplot as plt
imgs = [img for img in os.listdir(r"REDACTED")][1:]
cnt = 0
blurs = []
images = []
for img in imgs:
    path = os.path.join(r"REDACTED",img)
    image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    images.append(image)
    print(image.shape)
    print("image",cnt)
    blur_map = blur_detector.detectBlur(image, downsampling_factor=1, show_progress = False, num_scales=4, scale_start=2, num_iterations_RF_filter=3)
    blurs.append(blur_map)
    plt.imshow(image,cmap="grey")
    plt.imshow(blurs[cnt],alpha = 0.6)
    cnt += 1
    plt.show()

In [ ]:
import cv2
import numpy as np

def align_images_sift(img1, img2, idx):
    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Find keypoints and descriptors using SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Use FLANN matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Ratio test to find good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    # Minimum number of matches to find homography
    MIN_MATCH_COUNT = 10
    if len(good_matches) >= MIN_MATCH_COUNT:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Find homography matrix and mask
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        aligned_img = cv2.warpPerspective(img1, H, (img2.shape[1], img2.shape[0]))
        aligned_img = cv2.warpPerspective(blurs[idx], H, (img2.shape[1], img2.shape[0]))

        return aligned_img, H

    else:
        raise ValueError("Not enough matches are found - %d/%d" % (len(good_matches), MIN_MATCH_COUNT))

base = image[0]
contour = np.zeros((480,640))
depths = [0,0,0.13,0.01,0.01,0.1,0.07,0,0,0,0.07,0.03,0.01,0.02]
depths = np.array(depths)*1.2
for i in range(len(images)):
    if i == 2 or i == len(images)-1:
        continue
    try:
        aligned_img, H = align_images_sift(images[i],images[0],i)
    except:
        pass
    # print(aligned_img.shape)
    print(depths[i],aligned_img)
    # contour = np.maximum(aligned_img*depths[i],contour)
    min_val = np.min(aligned_img)
    max_val = np.max(aligned_img)

    # Step 2: Normalize to range [0, 0.05]
    aligned_img = 0.05 * (aligned_img - min_val) / (max_val - min_val)
    contour = np.maximum(contour,depths[i]*(aligned_img>0.01)+aligned_img*3*(aligned_img>0.01))
    # contour += np.maximum(contour,depths[i]+aligned_img/100)
    # plt.colorbar()
    plt.imshow(depths[i]+aligned_img/10)
    print(np.max(depths[i]+aligned_img/10))
    plt.show()
# ! the first image is the one being aligned with
# plt.imshow(images[1])
# plt.imshow(aligned_img,alpha = 0.6)


In [ ]:
contour

In [ ]:


def extract_contour(contour):
    xs = []
    ys = []
    zs = []
    cs = []
    thresh = 0.01
    for i in range(contour.shape[0]):
        for j in range(contour.shape[1]):
            if contour[i][j] > thresh and images[0][i][j] < 100:
                xs.append(i)
                ys.append(j)
                zs.append(contour[i][j])
                # print(image[0][i][j])
                cs.append(images[0][i][j])
    plt.imshow(contour)
    return cs,xs,ys,zs

cs,xs,ys,zs = extract_contour(contour)
print(len(xs))

In [ ]:
import plotly.graph_objs as go
import numpy as np

# Generate random 3D points

# xs = np.random.randn(100)
# ys = np.random.randn(100)
# zs = np.random.randn(100)

np.random.seed(0)
n_points = 100

# Create a trace
trace = go.Scatter3d(
    x=xs,
    y=ys,
    z=zs,
    mode='markers',
    marker=dict(
        size=2,    
        color = zs,# set color to an array/list of desired values
        opacity=0.4
    )
)

# Create layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X Axis',visible=False),
        yaxis=dict(title='Y Axis',visible=False),
        zaxis=dict(range = [0,2],title='Z Axis',visible=False),
    ),
    margin=dict(l=0, r=0, b=0, t=0)
)

# Create a figure
fig = go.Figure(data=[trace], layout=layout)
# fig.update_xaxes(visible=False)
# fig.update_yaxes(visible=False)

# Plot the figure
fig.show()


In [ ]:
# Create a figure and axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot 3D scatter plot
scatter = ax.scatter(xs, ys, zs)

# Set labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('3D Scatter Plot')

# Show the plot
plt.show()


In [ ]:
import numpy as np
from scipy.spatial import Delaunay
import plotly.graph_objects as go

# Step 1: Generate or load 3D points
# For this example, we'll generate random 3D points
np.random.seed(0)
points = np.stack((np.array(xs),np.array(ys),np.array(zs)),axis = 1)

# Step 2: Perform Delaunay Triangulation
tri = Delaunay(points)

# Extract the simplices (triangles/tetrahedrons)
simplices = tri.simplices

# Step 3: Visualize the mesh
# Create a list of Mesh3d elements for Plotly
mesh3d = go.Mesh3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    i=simplices[:, 0],
    j=simplices[:, 1],
    k=simplices[:, 2],
    opacity=0.01,
    color='blue'
)

# Create a scatter plot of the points for better visualization

def tetrahedron_volume(p1, p2, p3, p4):
    return np.abs(np.dot(p1 - p4, np.cross(p2 - p4, p3 - p4))) / 6.0

scatter3d = go.Scatter3d(
    x=points[:, 0],
    y=points[:, 1],
    z=points[:, 2],
    mode='markers',
    marker=dict(size=0.1, color='red')
)

# Create the figure and add the Mesh3d and Scatter3d
fig = go.Figure(data=[mesh3d])

# Update layout for better visualization
fig.update_layout(scene=dict(
                    xaxis=dict(visible=False),  # Range for x-axis
                    yaxis=dict(visible=False),  # Range for y-axis
                    zaxis=dict(range=[0, 1],visible=False)  # Range for z-axis
                ))

# Show the plot
fig.show()


In [ ]:
points = np.stack((np.array(xs),np.array(ys),np.array(zs)),axis = 1)
points.shape

In [ ]:
cv2.imwrit

In [ ]:
blur_map_normalized = cv2.normalize(blur_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
blur_map_colored = cv2.applyColorMap(blur_map_normalized, cv2.COLORMAP_JET)

        # Blend the original image and the colored depth map
blended_image = cv2.addWeighted(flipped, 0.6, blur_map_colored, 0.4, 0)
plt.imshow(blended_image)

In [ ]:
blur_map_colored.shape

In [ ]:
flipped.shape

In [ ]:
import cv2
import numpy as np

def variance_of_laplacian(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def get_blur(image):
    kernel_size = 3

# def get_blurriness(image):
    

# Load image
image = cv2.imread(r'C:\Users\victo\Desktop\Files\Tech\Code\Python\neurips\captures\image0.jpg')
print(image.shape)
# cv2.imshow('Most Blurry Section', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
from matplotlib import pyplot as plt
import blur_detector

image = cv2.imread(r'C:\Users\victo\Desktop\Files\Tech\Code\Python\neurips\captures\image0.jpg',0)
blur_map = blur_detector.detectBlur(image, downsampling_factor=1, num_scales=4, scale_start=2, num_iterations_RF_filter=3, show_progress=True)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.imshow(blur_map,alpha=0.6)
